<a href="https://colab.research.google.com/github/Moe-Zbeeb/Safety-Driven-Self-Compressing-Neural-Networks/blob/main/exp3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import getpass
import os


os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain.chat_models import ChatOpenAI


llm = ChatOpenAI(model="gpt-4o")

··········


<ipython-input-7-51802d1aa368>:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model="gpt-4o")


In [ ]:
from langchain.document_loaders import PyPDFLoader


In [40]:
loader = PyPDFLoader("/content/Chapter+16+vanLoon+-+Water+Pollution+and+Waste-Water+Treatment-23 2.pdf")

# Load the document into Langchain
docs = loader.load()

# Test by printing the content or the first chunk of the document
print(docs[0].page_content)

American University of BeirutWater Pollution & Wastewater TreatmentEnvironmentalChemistry, vanLoon& Duffy –Chapter16Mohamad Hmadeh, PhDAssistant ProfessorDepartment of Chemistry, Room 424Ext. 4988 , E-mail: mohamad.hmadeh@aub.edu.lb


In [38]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate

# Define prompt
prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a concise summary of the following:\\n\\n{context}")]
)

# Instantiate chain
chain = create_stuff_documents_chain(llm, prompt)

# Invoke chain
result1 = chain.invoke({"context": docs})
print(docs)


[Document(page_content='The chapter on water pollution and wastewater treatment from "Environmental Chemistry" by van Loon and Duffy, presented by Dr. Mohamad Hmadeh of the American University of Beirut, outlines key concepts and processes related to water pollution and treatment. Water pollution is defined as the presence of harmful chemicals or microorganisms above natural levels due to human activities, and pollutants are substances that adversely alter the environment. Criteria for distinguishing polluted from non-polluted water include physical, chemical, radiological, and microbiological properties.\n\nPoint sources of water pollutants are specific discharge points like factories, while nonpoint sources spread over larger areas, such as agricultural land. General types of water pollution include pathogens, oxygen-consuming agents, plant nutrients, toxic substances, dissolved solids, acids, suspended solids, oil, radioactive substances, and thermal pollution.\n\nBiological oxygen 

In [14]:


# Define prompt
prompt = ChatPromptTemplate.from_messages(
    [("system", "Extract 10 to 15 keywords that summarizes key concepts in the doc:\\n\\n{context}")]
)

# Instantiate chain
chain = create_stuff_documents_chain(llm, prompt)

# Invoke chain
result2 = chain.invoke({"context": docs})
print(result2)

1. Water Pollution
2. Wastewater Treatment
3. Environmental Chemistry
4. Pollution Criteria
5. Water Quality Guidelines
6. Point Sources
7. Nonpoint Sources
8. Biological Oxygen Demand (BOD)
9. Oxygen-Consuming Agents
10. Eutrophication
11. Nitrogen and Phosphorus Sources
12. Primary Water Treatment
13. Secondary Water Treatment
14. Tertiary Water Treatment
15. Sludge Disposal


In [15]:
context = result1 + "\n\n" + result2

In [16]:
type(context)

str

In [18]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import Document

# Define prompt
prompt = ChatPromptTemplate.from_messages(
    [("system", "Based on the summary:\n\n{context}, create an 8-question quiz that tests the reader's understanding of the document make the questions MCQ.")]
)

# Instantiate the chain
chain = create_stuff_documents_chain(llm, prompt)

# Check if context is a list
if isinstance(context, list):
    # Ensure each element in the list is a string and convert them to Document objects
    docs = [Document(page_content=item) for item in context]
else:
    # If context is not a list, convert it to a single Document
    docs = [Document(page_content=str(context))]

# Invoke the chain
result = chain.invoke({"context": docs})

# Print the result
#write to file
with open('quiz1.md', 'w') as f:
    f.write(result)
print(result)


Sure! Here are 8 multiple-choice questions (MCQs) based on the chapter summary:

1. **What is water pollution?**
   - A) The natural presence of chemicals in water
   - B) The presence of harmful chemicals or microorganisms above natural levels due to human activities
   - C) The accumulation of beneficial substances in water
   - D) The excess growth of aquatic plants

2. **Which of the following is a point source of water pollution?**
   - A) Agricultural runoff
   - B) Urban stormwater
   - C) A factory discharge pipe
   - D) Atmospheric deposition

3. **What does Biological Oxygen Demand (BOD) measure?**
   - A) The amount of dissolved oxygen in water
   - B) The oxygen required to oxidize organic carbon in water
   - C) The concentration of pathogens in water
   - D) The temperature of water

4. **What is eutrophication primarily caused by?**
   - A) High levels of dissolved oxygen
   - B) Excessive nutrients like nitrogen and phosphorus
   - C) Low Biological Oxygen Demand (BOD)


In [19]:
context += "\n\n" + "QUESTIONS"+ '\n' + result
print(type(context))

<class 'str'>


In [20]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import Document

# Define prompt for the chain
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Based on the following summary: {context} and the quiz questions: {{result}}, answer each of the following questions concisely and return only the answers.")
    ]
)

# Instantiate chain with the defined prompt
chain = create_stuff_documents_chain(llm, prompt)

# Check and convert the context to a Document object
if isinstance(context, list):
    # If context is a list, join its elements into a single string and create a Document
    context = " ".join(context)  # Combine list into a single string
context_doc = Document(page_content=context)  # Wrap the context in a Document

# Print the type to ensure it is now a Document
print(type(context_doc))  # This should print: <class 'langchain.schema.Document'>

# Invoke chain with the context now wrapped in a Document object
answers = chain.invoke({"context": [context_doc]})  # Pass as a list of Document objects

# Print the generated answers
print(answers)


<class 'langchain_core.documents.base.Document'>
1. B
2. C
3. B
4. B
5. C
6. A
7. B
8. C


In [21]:
answers

'1. B\n2. C\n3. B\n4. B\n5. C\n6. A\n7. B\n8. C'

In [22]:
#make the string as 1d list
answers = answers.split("\n")
print(answers)

['1. B', '2. C', '3. B', '4. B', '5. C', '6. A', '7. B', '8. C']


In [23]:
for i in range(len(answers)):
    answers[i] = answers[i][3]
    print(answers[i])

B
C
B
B
C
A
B
C


In [24]:
answers

['B', 'C', 'B', 'B', 'C', 'A', 'B', 'C']

In [29]:
inputuser = ['A', 'C', 'B', 'B', 'C', 'A', 'B', 'C']

#grader
def grade(answers, inputuser):
    wrongs = []
    score = 0
    for i in range(len(answers)):
        if answers[i] != inputuser[i]:
            wrongs.append(i)
    return wrongs
print(grade(answers, inputuser))

[0]


In [30]:
# Get the wrong answers indices from grade
wrongs = grade(answers, inputuser)  # e.g., [1, 2]

# Update context with wrong answers information
context = context + '\n' + "WRONG ANSWERS\n" + str(wrongs)  # Add wrong answers info to context

# Define prompt for generating new questions based on wrong answers
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Based on the following details: {context} and the quiz questions, "
                   "the user made mistakes in the questions with indices {wrongs}. Create additional questions "
                   "in the same style to assess the student further on the topics they struggled with.")
    ]
)

# Instantiate chain with the defined prompt
chain = create_stuff_documents_chain(llm, prompt)

# Convert context to a Document object if necessary
if isinstance(context, list):
    context = " ".join(context)  # If context is a list, combine it into a single string
context_doc = Document(page_content=context)  # Wrap the context in a Document object

# Print the type to ensure it is now a Document
print(type(context_doc))  # This should print: <class 'langchain.schema.Document'>

# Invoke chain with the context now wrapped in a Document object
questions2 = chain.invoke({"context": [context_doc], "result": context_doc, "wrongs": str(wrongs)})  # Pass context, result, and wrongs

# Print the generated answers
#write to file
with open('quiz2.md', 'w') as f:
    f.write(questions2)
print(questions2)

<class 'langchain_core.documents.base.Document'>
Sure! Here are two additional questions to further assess the student's understanding of the topics where mistakes were made:

**Additional Questions:**

1. **What are the criteria used to distinguish polluted water from non-polluted water?**
   - A) Only chemical properties
   - B) Only physical properties
   - C) Physical, chemical, radiological, and microbiological properties
   - D) Only microbiological properties

2. **Which of the following is NOT a general type of water pollution?**
   - A) Pathogens
   - B) Dissolved solids
   - C) Heavy metals
   - D) Thermal pollution

**Answers:**
1. C) Physical, chemical, radiological, and microbiological properties
2. C) Heavy metals


In [31]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import Document

# Combine context with additional questions
new_context = context + '\n' + "QUESTIONS2\n" + questions2

# Define the prompt to return only the letter corresponding to the correct answer
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Based on the following summary: {context}, answer each of the following questions under the section 'QUESTIONS2'. "
                   "Return only the letter of the correct answer (A, B, C, or D) with no additional information.")
    ]
)

# Instantiate the chain with the defined prompt
chain = create_stuff_documents_chain(llm, prompt)

# Ensure context is a string before wrapping it in Document
if isinstance(new_context, list):
    new_context = " ".join(new_context)  # Convert list to a single string

# Wrap the new context in a Document object
new_context_doc = Document(page_content=new_context)  # Wrap the new context in a Document

# Print the type to ensure it is now a Document
print(type(new_context_doc))  # This should print: <class 'langchain.schema.Document'>

# Invoke the chain with the new context now wrapped in a Document object
answers = chain.invoke({"context": [new_context_doc]})  # Pass as a list of Document objects

# Print the generated answers (it should return only the alphabet of the answer)
print(answers)


<class 'langchain_core.documents.base.Document'>
1. C
2. C


In [ ]:
#make the string as 1d list
answers = answers.split("\n")
print(answers)

['1. B', '2. B', '3. B', '4. B', '5. B', '6. B']


In [ ]:
inputuser = ['b', 'c', 'b', 'c', 'b', 'c', 'b', 'b']

#grader
def grade(answers, inputuser):
    wrongs = []
    score = 0
    for i in range(len(answers)):
        if answers[i] != inputuser[i]:
            wrongs.append(i)
    return wrongs
print(grade(answers, inputuser))

[0, 1, 2, 3, 4, 5]


In [45]:
complete_context = new_context + '\n' + "WRONG ANSWERS\n"
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Based on the following details: {context}, Your task is to assess the my knowledge based on my answers. "
                   "In your evaluation report, please include the following:"
                   "Identify any topics where the I seem confused or mixing concepts."
                   "Highlight the areas where the I demonstrated strong understanding."
                   "Point out any areas where the I showed weaknesses or lacks knowledge")

    ]
)
# Step 4: Instantiate chain with the defined prompt
chain = create_stuff_documents_chain(llm, prompt)

# Step 5: Convert the complete context to a Document object
Assesment_doc = Document(page_content=complete_context)

# Print the type to ensure it is now a Document
print(type(Assesment_doc))  # This should print: <class 'langchain.schema.Document'>

# Step 6: Invoke chain to generate Final Assesment
Assesment = chain.invoke({"context": [Assesment_doc]})
print("Final Assesment", Assesment)
with open("Final_Assesment.md", "w") as file:
    file.write(Assesment)
    file.close()



<class 'langchain_core.documents.base.Document'>
Final Assesment ### Evaluation Report

#### Areas of Strong Understanding

1. **Definition of Water Pollution:** 
   - You correctly identified water pollution as the presence of harmful chemicals or microorganisms above natural levels due to human activities.
   
2. **Point Source of Water Pollution:**
   - You correctly recognized a factory discharge pipe as a point source of water pollution.

3. **Biological Oxygen Demand (BOD):**
   - You accurately understood that BOD measures the oxygen required to oxidize organic carbon in water.

4. **Eutrophication:**
   - You correctly identified that eutrophication is primarily caused by excessive nutrients like nitrogen and phosphorus.

5. **Disinfection Method:**
   - You correctly identified that chlorine is a disinfection method that involves the use of a potentially hazardous gas.

6. **Home Water Softening:**
   - You correctly recognized ion-exchange as a common process used in home wat

In [ ]:
complete_context = new_context + '\n' + "WRONG ANSWERS\n"


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Based on the following details: {context}, generate 10 to 20 flashcards in the format of 'Question: Answer:', "
                   "a small summary of the key topics (5-10 lines), and a study plan to exit the exam successfully. "
                   "The study plan should prioritize key areas where the student needs improvement and help them focus efficiently.")
    ]
)

# Step 4: Instantiate chain with the defined prompt
chain = create_stuff_documents_chain(llm, prompt)

# Step 5: Convert the complete context to a Document object
complete_context_doc = Document(page_content=complete_context)

# Print the type to ensure it is now a Document
print(type(complete_context_doc))  # This should print: <class 'langchain.schema.Document'>

# Step 6: Invoke chain to generate flashcards, summary, and study plan
study_material = chain.invoke({"context": [complete_context_doc]})

# Step 7: Print the generated flashcards, summary, and study plan
print("Generated Study Material:\n", study_material)
#write the study plan to a file
with open("study_plan.md", "w") as file:
    file.write(study_material)
    file.close()

<class 'langchain_core.documents.base.Document'>
Generated Study Material:
 ### Flashcards

1. **Question:** What is the primary objective of the method proposed in "Safety Driven Self Compressing Neural Networks"?
   **Answer:** To reduce the size of neural networks by eliminating redundant weights and minimizing bit depth.

2. **Question:** How does the proposed method maintain floating-point accuracy while reducing model size?
   **Answer:** By using a fully differentiable quantization function and the Straight-Through Estimator (STE).

3. **Question:** What are the key advantages of the proposed method?
   **Answer:** Fewer weights, efficient bit usage, faster training and execution, and a simplified design process.

4. **Question:** Why does the proposed method group weights by channel?
   **Answer:** To enhance stability and avoid forgetting important features.

5. **Question:** What role does the custom loss function play in the proposed method?
   **Answer:** It balances task a

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document

# Step 1: Define a more powerful prompt template with detailed instructions
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """Take the following study material {context} and summarize it in 100-150 lines, highlighting only the most critical topics and concepts to ensure exam success. Focus on the following:

1. Comprehensive explanations of definitions and core principles. Expand on why they are important.
2. In-depth explanation of key formulas, equations, or models, with step-by-step reasoning and how they apply in problem-solving.
3. Diagrams or visual representations should be described in a way that helps retain crucial concepts.
4. Analyze common mistakes students make and provide tips on how to avoid them.
5. Provide detailed real-world applications or examples for each core concept.
6. Structure the summary logically, ensuring it is easy to follow.

Make the return elaborated where you explain details and notations in a well manner approach
Be sure to elaborate on each point and ensure clarity. Keep the response length between 100 to 150 lines. The summary should be highly detailed, professional, and suitable for advanced exam preparation.""")
    ]
)

# Step 2: Instantiate the chain with the more powerful prompt
chain = create_stuff_documents_chain(llm, prompt)

# Step 3: Convert the complete context to a Document object
complete_context_doc = Document(page_content=complete_context)

# Verify that it's a Document object
print(type(complete_context_doc))  # Should print <class 'langchain.schema.Document'>

# Step 4: Invoke the chain to generate in-depth study material
study_material2 = chain.invoke({"context": [complete_context_doc]})

# Step 5: Print the generated study material (flashcards, summary, and study plan)
print("Generated Study Material:\n", study_material2)

# Step 6: Write the study material to a markdown file
with open("study_plan2.md", "w") as file:
    file.write(study_material2)  # Save generated material in study_plan2.md


<class 'langchain_core.documents.base.Document'>
Generated Study Material:
 The paper "Safety Driven Self Compressing Neural Networks" by Mohammad Zbeeb introduces a novel method for reducing the size of neural networks while maintaining their accuracy and performance. This method focuses on eliminating redundant weights and minimizing the bit depth required to represent the remaining weights. Here is a detailed summary highlighting the critical topics and concepts:

### 1. Core Principles and Definitions

**Neural Network Size Reduction**:
- **Definition**: The process of decreasing the number of parameters (weights) in a neural network to make it smaller and more efficient.
- **Importance**: Smaller neural networks require less computational power and memory, making them suitable for deployment in resource-constrained environments such as mobile devices and edge computing.

**Redundant Weights**:
- **Definition**: Weights in a neural network that do not significantly contribute to th